In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "lpN0n7L5GkN776GG51v4MKGAk"
    consumer_secret = "fVmvM0z3X4e3pQZCyEErutUeeWMJJbMqFSloqryIOa9dttwGHh"
    access_token = "1084744810387300352-v7kC8jFV1a9Iq13WknbRDi6T09egtg"
    access_token_secret = "VmNeWJwgirU0wNxK0qGoltLQmruxjT9fR0grQK16JHkbC"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    # LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
    #              -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
    #              -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    LOCATIONS = [129.408463169, 31.0295791692, 145.543137242, 45.5514834662]

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517671206849507328, 'username': 'YogannaNupur', 'created_at': 'Sat Apr 23 01:06:27 +0000 2022', 'lng': 139.931051, 'lat': 35.715682, 'text': 'Good morning my lovelies 💞 it’s a not so sunny Saturday in Tokyo, woke up late but ready for yoga and meditation. L… https://t.co/V0GRHer0Nk'}
{'id': 1517671326164852736, 'username': 'Erika91000000', 'created_at': 'Sat Apr 23 01:06:55 +0000 2022', 'lng': 139.745379, 'lat': 35.652789, 'text': 'Awesome'}
{'id': 1517671600954687489, 'username': 'KO_VintageTokyo', 'created_at': 'Sat Apr 23 01:08:01 +0000 2022', 'lng': 139.69262600000002, 'lat': 35.666851, 'text': 'Condition is A1. DM to purchase \U0001f90e https://t.co/eC0LxInUUQ'}
{'id': 1517671629593411585, 'username': 'JQ2KPP', 'created_at': 'Sat Apr 23 01:08:08 +0000 2022', 'lng': 138.684687, 'lat': 35.2373305, 'text': '@womensoudan 🎶𝙜𝙤𝙤𝙤𝙤𝙙 𝙢𝙤𝙧𝙣𝙞𝙣𝙜!🎶'}
{'id': 1517671650614861824, 'username': 'potaspotja', 'created_at': 'Sat Apr 23 01:08:13 +0000 2022', 'lng': 140.058, 'lat': 35.7685, 'tex